In [1]:
# koneksi drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# buat folder
!pip install python-docx
import os

folder_path = "/content/drive/MyDrive/eddsa_document"
test_path = "/content/drive/MyDrive/test_eddsa_document"

os.makedirs(folder_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)
print(f"Folder dibuat di: {folder_path}")
print(f"Folder dibuat di: {test_path}")

Folder dibuat di: /content/drive/MyDrive/eddsa_document
Folder dibuat di: /content/drive/MyDrive/test_eddsa_document


In [35]:
# upload doc
from google.colab import files
import shutil

uploaded = files.upload()

for filename in uploaded.keys():
    shutil.move(filename, os.path.join(folder_path, filename))
    print(f"{filename} dipindahkan ke {folder_path}")

Saving Materi2.docx to Materi2.docx
Materi2.docx dipindahkan ke /content/drive/MyDrive/eddsa_document


In [36]:
# baca doc
from docx import Document

filepath = "/content/drive/MyDrive/eddsa_document/Materi2.docx"

doc = Document(filepath)
text = "\n".join([p.text for p in doc.paragraphs])
print(text)



Latihan

Kasus 1
Disajikan data sebagai berikut:


Buatlah tabel distribusi frekuensi komulatif, gambar poligon dan histogram, serta kurva ogivenya.

Kasus 2
Berikut data penjualan (dalam unit) dari 60 toko komputer pada acara pameran komputer, penjualan tertinggi oleh beberapa toko komputer adalah 78 unit, dari tabel tersebut:
Buatlah tabel distribusi frekuensinya







Jawaban

Kasus 1
K 	= 1+3,322*logN 
	= 1+3,322*log40 
	= 1+3,322(1,60) 
	= 6,3152 
	= 6 
R	= 159 – 100 
	= 59
Ci 	= 59/6 = 9,833 
	= 10

Tabel Distributif Frekuensi: 















Kasus 2
Tabel Distributif Frekuensi: 



In [37]:
# extract doc
import zipfile

docx_path = "/content/drive/MyDrive/eddsa_document/Materi2.docx"
extract_to = "/content/drive/MyDrive/eddsa_document/output_document/"

with zipfile.ZipFile(docx_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

with open("/content/drive/MyDrive/eddsa_document/output_document/word/document.xml", "r", encoding="utf-8") as f:
    isi = f.read()
    print(isi)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<w:document xmlns:wpc="http://schemas.microsoft.com/office/word/2010/wordprocessingCanvas" xmlns:cx="http://schemas.microsoft.com/office/drawing/2014/chartex" xmlns:cx1="http://schemas.microsoft.com/office/drawing/2015/9/8/chartex" xmlns:cx2="http://schemas.microsoft.com/office/drawing/2015/10/21/chartex" xmlns:cx3="http://schemas.microsoft.com/office/drawing/2016/5/9/chartex" xmlns:cx4="http://schemas.microsoft.com/office/drawing/2016/5/10/chartex" xmlns:cx5="http://schemas.microsoft.com/office/drawing/2016/5/11/chartex" xmlns:cx6="http://schemas.microsoft.com/office/drawing/2016/5/12/chartex" xmlns:cx7="http://schemas.microsoft.com/office/drawing/2016/5/13/chartex" xmlns:cx8="http://schemas.microsoft.com/office/drawing/2016/5/14/chartex" xmlns:mc="http://schemas.openxmlformats.org/markup-compatibility/2006" xmlns:aink="http://schemas.microsoft.com/office/drawing/2016/ink" xmlns:am3d="http://schemas.microsoft.com/office/drawing/2

In [38]:
# hash
import hashlib

xml_bytes = isi.encode('utf-8')
doc_hash = hashlib.sha256(xml_bytes).digest()

print("Hash dokumen (SHA-256):", doc_hash.hex())

Hash dokumen (SHA-256): e9a924de255e81e0462f2a64613580cc5a90024cdbfac17d0e2fe4f668f4ead9


In [39]:
# kriptografi
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey

private_key = Ed25519PrivateKey.generate()
public_key = private_key.public_key()

signature = private_key.sign(doc_hash)

print("Signature (hex):", signature.hex())

Signature (hex): 5d980ceea144cfaf7a8d33a167ac72dbfd6e58e59cee0745420a759172e9f0b74aa548f18bc494e5e4b032ebe79f1b5fd60cec63fb6aabdc1ef09e868e33d60a


In [40]:
# buat signature key & public key
from cryptography.hazmat.primitives import serialization

signature_path = "/content/drive/MyDrive/eddsa_document/signature.sig"
with open(signature_path, "wb") as f:
    f.write(signature)

public_key_path = "/content/drive/MyDrive/eddsa_document/public_key.pem"
with open(public_key_path, "wb") as f:
    f.write(public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ))

print(f"Signature tersimpan di {signature_path}")
print(f"Public key tersimpan di {public_key_path}")

Signature tersimpan di /content/drive/MyDrive/eddsa_document/signature.sig
Public key tersimpan di /content/drive/MyDrive/eddsa_document/public_key.pem


In [41]:
# buat kembali doc
import os
import zipfile

def zip_dir(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, folder_path)
                zipf.write(full_path, rel_path)

folder_ekstrak = "/content/drive/MyDrive/eddsa_document/output_document/"
output_docx_path = "/content/drive/MyDrive/eddsa_document/Materi2New.docx"

zip_dir(folder_ekstrak, output_docx_path)

print(f"File DOCX baru berhasil dibuat: {output_docx_path}")

File DOCX baru berhasil dibuat: /content/drive/MyDrive/eddsa_document/Materi2New.docx


In [43]:
# ==== BAGIAN UJI (ganti seluruh blok uji lama dengan ini) ====
import zipfile
import hashlib
from cryptography.hazmat.primitives import serialization
from cryptography.exceptions import InvalidSignature
import shutil
from google.colab import files
import os

# 1) upload file (sama seperti sebelumnya)
uploaded = files.upload()
for filename in uploaded.keys():
    shutil.move(filename, os.path.join(test_path, filename))
    print(f"{filename} dipindahkan ke {test_path}")

# 2) baca content.xml dari docx uji sebagai TEXT (sama method dg sign sebelumnya)
docx_path = "/content/drive/MyDrive/test_eddsa_document/Materi2New.docx"
extract_to = "/content/drive/MyDrive/test_eddsa_document/output_document_test/"

with zipfile.ZipFile(docx_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

with open(os.path.join(extract_to, "word/document.xml"), "r", encoding="utf-8") as f:
    xml_content = f.read()

# compute hash the same way you did saat sign
hash_value = hashlib.sha256(xml_content.encode('utf-8')).digest()
print("sha256 (test):", hashlib.sha256(xml_content.encode('utf-8')).hexdigest())

# 3) load public key (yang disimpan saat sign)
with open("/content/drive/MyDrive/eddsa_document/public_key.pem", "rb") as f:
    public_key = serialization.load_pem_public_key(f.read())

# 4) load signature YANG SUDAH ADA (jANGAN generate private key baru!)
signature_path_original = "/content/drive/MyDrive/eddsa_document/signature.sig"
with open(signature_path_original, "rb") as f:
    signature = f.read()

# 5) verifikasi (public_key.verify akan lempar InvalidSignature bila gagal)
try:
    public_key.verify(signature, hash_value)
    print("✅ Signature valid. Dokumen asli dan belum diubah.")
except InvalidSignature:
    print("❌ Signature tidak valid! Dokumen sudah diubah atau signature bukan berasal dari pasangan public key ini.")
except Exception as e:
    print("❌ Error saat verifikasi:", e)

Saving Materi2New.docx to Materi2New.docx
Materi2New.docx dipindahkan ke /content/drive/MyDrive/test_eddsa_document
sha256 (test): e750cd1b91a49d6c093db704464853a9f26427ccec5ef0cd820dbac174b23979
❌ Signature tidak valid! Dokumen sudah diubah atau signature bukan berasal dari pasangan public key ini.
